In [1]:
import pandas as pd
import numpy as np
import csv
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
tol_mz = 0.002
tol_ret = 0.2

In [3]:
# data_lib = pd.read_csv('../data/Lemas_POS_LIBRARY_MATCH_exported.csv')
data_lib = pd.read_csv('../data/Positive_Garrett_MetaboliteStd_Library_RP_edited01152019JG.csv')

In [4]:
#data_meta1 = pd.read_csv('../data/milk_data_pos_ph_summaries_filter_allgroups_threshold_100.csv')

In [5]:
# This is the data file that needs to be searched with the database
data_meta2 = pd.read_csv('../data/Lemas_POS_mzmine_ht_17DEC18_data.csv')

In [6]:
len(data_lib)

631

In [7]:
#len(data_meta1)

In [8]:
len(data_meta2)

6841

In [9]:
data_lib.head()

,Garrett Lab ID,M/Z,Retention,Name,Formula,Molecular Weight,Adduct,Notes,Unnamed: 8,KEGG,Unnamed: 10
0,428p,75.09170,0.620,"1,3-DIAMINOPROPANE",C3H10N2,74.12490,NaN,NaN,NaN,C00986,NaN
1,750p,76.03930,0.736,GLYCINE,C2H5NO2,75.06660,NaN,NaN,NaN,C00037,NaN
2,2723790p,77.01680,0.908,THIOUREA,CH4N2S,76.12086,NaN,NaN,NaN,C14415,NaN
3,1049p,80.04971,0.973,Pyridine,C5H5N,79.04220,NaN,Shaddox Samples - Compound Discoverer,NaN,C00747,NaN
4,1045p,89.10730,0.580,PUTRESCINE,C4H12N2,88.15148,NaN,RT corrected from 0.63min by std 03.01.2018 CC,NaN,C00134,NaN


## drop NaN for the lib data (Do not run the following with Garrett Lib)

In [11]:
# data_lib.dropna(subset = ['ID', 'Identification method'], how = 'all', inplace = True)

In [12]:
len(data_lib)

631

In [13]:
data_lib.head(10)

,Garrett Lab ID,M/Z,Retention,Name,Formula,Molecular Weight,Adduct,Notes,Unnamed: 8,KEGG,Unnamed: 10
0,428p,75.09170,0.620,"1,3-DIAMINOPROPANE",C3H10N2,74.12490,NaN,NaN,NaN,C00986,NaN
1,750p,76.03930,0.736,GLYCINE,C2H5NO2,75.06660,NaN,NaN,NaN,C00037,NaN
2,2723790p,77.01680,0.908,THIOUREA,CH4N2S,76.12086,NaN,NaN,NaN,C14415,NaN
3,1049p,80.04971,0.973,Pyridine,C5H5N,79.04220,NaN,Shaddox Samples - Compound Discoverer,NaN,C00747,NaN
4,1045p,89.10730,0.580,PUTRESCINE,C4H12N2,88.15148,NaN,RT corrected from 0.63min by std 03.01.2018 CC,NaN,C00134,NaN
5,ISO-01p,90.05495,0.751,ALANINE/SARCOSINE,C3H7NO2,89.09318,NaN,NaN,NaN,NaN,NaN
6,NaN,90.05495,9.000,N-BOC-L-Alanine,C8H15NO4,NaN,NaN,Injection Standard,NaN,NaN,NaN
7,612p,91.03953,0.840,L-(+)-Lactic Acid,C3H6O3,90.07794,NaN,NaN,NaN,NaN,NaN
8,IntStd9p,93.06556,0.720,L-Alanine 13C3,*CH3*CH(NH2)*COOH,NaN,NaN,Cambridge Isotope Project,NaN,NaN,NaN
9,8871p,96.04440,3.310,2-HYDROXYPYRIDINE,C5H5NO,95.09930,NaN,NaN,NaN,C02502,NaN


In [41]:
data_lib['row ID'] = 'rowID_' + data_lib['row ID'].astype(str)

In [45]:
data_lib.rename(index = str, columns = {'row ID': 'row_ID'}, inplace = True)

In [46]:
data_lib.head()

,row_ID,row m/z,row retention time,ID,Identification method,Molecular formula,Name
0,rowID_4114,127.039212,0.800112,237332p,Positive_Garrett_MetaboliteStd_Library_RP_edit...,C6H6O3,5-Hydroxymethyl-2-furaldehyde
1,rowID_2441,184.073311,0.714081,1014p,Positive_Garrett_MetaboliteStd_Library_RP_edit...,C5H14NO4P,PHOSPHOCHOLINE
2,rowID_2442,258.109964,0.752360,71920p,Positive_Garrett_MetaboliteStd_Library_RP_edit...,C8H20NO6P,Glycerophosphocholine
3,rowID_2122,162.112494,0.956648,10917p,Positive_Garrett_MetaboliteStd_Library_RP_edit...,C7H15NO3,L-CARNITINE
17,rowID_1711,132.076942,0.891458,586p,Positive_Garrett_MetaboliteStd_Library_RP_edit...,C4H9N3O2,CREATINE


## Merge dataframes

#### merge lib and data_meta1

In [137]:
data_meta1.head(10)

,row_ID,f_value,p_value_f,mean_fat,mean_skim,mean_whole,sd_fat,sd_skim,sd_whole,mean_skim_fat,mean_whole_skim,mean_whole_fat,p_value_skim_fat,p_value_whole_skim,p_value_whole_fat,flag_skim_fat,flag_whole_skim,flag_whole_fat
0,rowID_4114,13.305295,0.000901,155800000.0,2.768000e+08,2.534000e+08,5.939023e+07,2.950763e+07,1.567801e+07,121000000.0,-23400000.0,97600000.0,0.001048,0.626331,0.005316,1.0,0.0,1.0
1,rowID_8983,4.545111,0.033931,15082215.3,3.278000e+07,2.428000e+07,9.701753e+06,1.067694e+07,7.101549e+06,17697784.7,-8500000.0,9197784.7,0.027025,0.349019,0.296771,1.0,0.0,0.0
2,rowID_8894,2.890050,0.094511,134600000.0,2.642000e+08,2.138000e+08,9.620967e+07,7.673135e+07,8.371798e+07,129600000.0,-50400000.0,79200000.0,0.081729,0.634140,0.344588,0.0,0.0,0.0
3,rowID_2450,19.131604,0.000185,101560000.0,2.228000e+08,2.032000e+08,4.965927e+07,1.878031e+07,2.240982e+07,121240000.0,-19600000.0,101640000.0,0.000245,0.631695,0.001107,1.0,0.0,1.0
4,rowID_1,0.334151,0.722399,146400000.0,1.536000e+08,1.392400e+08,2.600577e+07,2.629258e+07,3.076667e+07,7200000.0,-14360000.0,-7160000.0,0.912212,0.699738,0.913134,0.0,0.0,0.0
5,rowID_9025,6.749691,0.010862,17507287.1,5.270000e+07,4.088000e+07,1.789419e+07,1.752555e+07,9.245107e+06,35192712.9,-11820000.0,23372712.9,0.009277,0.468620,0.079958,1.0,0.0,0.0
6,rowID_2441,3.645296,0.057945,755800000.0,1.244200e+09,1.155400e+09,3.842599e+08,2.633120e+08,2.480339e+08,488400000.0,-88800000.0,399600000.0,0.063131,0.890576,0.137298,0.0,0.0,0.0
7,rowID_2453,17.392114,0.000285,82320000.0,1.658000e+08,1.566000e+08,3.861356e+07,1.255787e+07,1.258173e+07,83480000.0,-9200000.0,74280000.0,0.000450,0.826575,0.001194,1.0,0.0,1.0
8,rowID_2442,7.229254,0.008704,468200000.0,7.486000e+08,6.994000e+08,1.931287e+08,5.745259e+07,7.689148e+07,280400000.0,-49200000.0,231200000.0,0.010133,0.809582,0.031089,1.0,0.0,1.0
9,rowID_2459,10.894095,0.002007,56940000.0,1.039200e+08,9.458000e+07,2.344297e+07,1.112349e+07,1.335410e+07,46980000.0,-9340000.0,37640000.0,0.002269,0.664591,0.010662,1.0,0.0,1.0


#### merge lib and data_meta2

In [15]:
data_meta2.head(10)

,row_ID,row_m_z,row_retention_time,flag_fat_threshold_005,flag_skim_threshold_005,flag_whole_threshold_005,flag_fat_threshold_100,flag_skim_threshold_100,flag_whole_threshold_100,flag_fat_threshold_200,flag_skim_threshold_200,flag_whole_threshold_200,row_ID_anova,f_value_anova,p_value_f_anova,mean_fat_anova,mean_skim_anova,mean_whole_anova,sd_fat_anova,sd_skim_anova,sd_whole_anova,mean_skim_fat_anova,mean_whole_skim_anova,mean_whole_fat_anova,p_value_skim_fat_anova,p_value_whole_skim_anova,p_value_whole_fat_anova,flag_skim_fat_anova,flag_whole_skim_anova,flag_whole_fat_anova,row_ID_kruskalwallis,k_value_kruskalwallis,p_value_k_kruskalwallis,mean_fat_kruskalwallis,mean_skim_kruskalwallis,mean_whole_kruskalwallis,sd_fat_kruskalwallis,sd_skim_kruskalwallis,sd_whole_kruskalwallis,mean_skim_fat_kruskalwallis,mean_whole_skim_kruskalwallis,mean_whole_fat_kruskalwallis,p_value_skim_fat_kruskalwallis,p_value_whole_skim_kruskalwallis,p_value_whole_fat_kruskalwallis,flag_skim_fat_kruskalwallis,flag_whole_skim_kruskalwallis,flag_whole_fat_kruskalwallis
0,rowID_2020,184.073424,0.732373,0,0,0,0,0,0,0,0,0,rowID_2020,1.684320,0.239135,787473504.0,1.209770e+09,1.155680e+09,4.580742e+08,2.841735e+08,2.926282e+08,422296112.0,-54089872.0,368206240.0,0.262054,0.974677,0.348732,0,0,0,rowID_2020,3.038462,0.218880,787473504.0,1.209770e+09,1.155680e+09,4.580742e+08,2.841735e+08,2.926282e+08,422296112.0,-54089872.0,368206240.0,0.148915,0.563703,0.148915,0,0,0
1,rowID_3609,258.109775,0.760621,0,0,0,0,0,0,0,0,0,rowID_3609,3.674578,0.068134,483369900.0,7.494516e+08,7.030400e+08,2.316422e+08,7.140548e+07,8.493822e+07,266081668.0,-46411600.0,219670068.0,0.074284,0.898812,0.145841,0,0,0,rowID_3609,2.807692,0.245650,483369900.0,7.494516e+08,7.030400e+08,2.316422e+08,7.140548e+07,8.493822e+07,266081668.0,-46411600.0,219670068.0,0.148915,0.386476,0.248213,0,0,0
2,rowID_1973,162.112531,0.945414,0,0,0,0,0,0,0,0,0,rowID_1973,6.138617,0.020820,448668172.0,8.627042e+08,8.278780e+08,2.609883e+08,1.191339e+08,1.441874e+08,414036068.0,-34826224.0,379209844.0,0.028245,0.961999,0.042720,1,0,1,rowID_1973,4.884615,0.086960,448668172.0,8.627042e+08,8.278780e+08,2.609883e+08,1.191339e+08,1.441874e+08,414036068.0,-34826224.0,379209844.0,0.083265,0.772830,0.043308,0,0,1
3,rowID_8149,204.123096,5.553267,0,0,0,0,0,0,0,0,0,rowID_8149,12.074785,0.002831,71236427.0,1.318934e+08,1.382231e+08,1.385184e+07,2.724531e+07,2.061812e+07,60656959.0,6329742.0,66986701.0,0.007522,0.908082,0.004091,1,0,1,rowID_8149,7.423077,0.024440,71236427.0,1.318934e+08,1.382231e+08,1.385184e+07,2.724531e+07,2.061812e+07,60656959.0,6329742.0,66986701.0,0.020921,0.772830,0.020921,1,0,1
4,rowID_2102,210.060849,1.830206,0,0,0,0,0,0,0,0,0,rowID_2102,11.726792,0.003115,174831080.0,3.741021e+08,3.945621e+08,7.925454e+07,8.211132e+07,4.532786e+07,199270976.0,20460032.0,219731008.0,0.008157,0.913131,0.004503,1,0,1,rowID_2102,7.538462,0.023070,174831080.0,3.741021e+08,3.945621e+08,7.925454e+07,8.211132e+07,4.532786e+07,199270976.0,20460032.0,219731008.0,0.020921,0.563703,0.020921,1,0,1
5,rowID_2498,366.139297,0.858138,0,0,0,0,0,0,0,0,0,rowID_2498,3.340064,0.082228,96487363.0,1.752444e+08,1.651902e+08,4.026520e+07,5.428550e+07,4.508629e+07,78757033.0,-10054184.0,68702849.0,0.095530,0.950884,0.151050,0,0,0,rowID_2498,4.307692,0.116037,96487363.0,1.752444e+08,1.651902e+08,4.026520e+07,5.428550e+07,4.508629e+07,78757033.0,-10054184.0,68702849.0,0.083265,0.563703,0.083265,0,0,0
6,rowID_2053,104.107469,0.879552,0,0,0,1,0,0,1,1,1,rowID_2053,2.459968,0.140515,298115892.0,5.241520e+08,5.016882e+08,8.216562e+07,2.089269e+08,1.588727e+08,226036100.0,-22463792.0,203572308.0,0.164608,0.978229,0.219718,0,0,0,rowID_2053,4.884615,0.086960,298115892.0,5.241520e+08,5.016882e+08,8.216562e+07,2.089269e+08,1.588727e+08,226036100.0,-22463792.0,203572308.0,0.043308,1.000000,0.083265,1,0,0
7,rowID_2012,132.076966,0.891071,0,0,0,0,0,0,0,0,0,rowID_2012,7.969972,0.010187,306063756.0,6.022965e+08,5.711609e+08,1.780364e+08,6.037563e+07,6.750942

In [16]:
data_meta2.shape

(697, 48)

In [26]:
d1 = data_lib[(data_lib['M/Z'] < data_meta2.row_m_z.iloc[0] + tol_mz) & 
         (data_lib['M/Z'] > data_meta2.row_m_z.iloc[0] - tol_mz) &
         (data_lib['Retention'] < data_meta2.row_retention_time.iloc[0] + tol_ret) &
         (data_lib['Retention'] > data_meta2.row_retention_time.iloc[0] - tol_ret)]

In [27]:
d1["merge_key"] = 0

In [28]:
d2 = data_meta2.iloc[[0]]

In [29]:
d2["merge_key"] = 0

In [30]:
d1

,Garrett Lab ID,M/Z,Retention,Name,Formula,Molecular Weight,Adduct,Notes,Unnamed: 8,KEGG,Unnamed: 10,merge_key
240,1014p,184.0733,0.77,PHOSPHOCHOLINE,C5H14NO4P,184.150662,NaN,NaN,NaN,C00588,NaN,0


In [31]:
d2

,row_ID,row_m_z,row_retention_time,flag_fat_threshold_005,flag_skim_threshold_005,flag_whole_threshold_005,flag_fat_threshold_100,flag_skim_threshold_100,flag_whole_threshold_100,flag_fat_threshold_200,flag_skim_threshold_200,flag_whole_threshold_200,row_ID_anova,f_value_anova,p_value_f_anova,mean_fat_anova,mean_skim_anova,mean_whole_anova,sd_fat_anova,sd_skim_anova,sd_whole_anova,mean_skim_fat_anova,mean_whole_skim_anova,mean_whole_fat_anova,p_value_skim_fat_anova,p_value_whole_skim_anova,p_value_whole_fat_anova,flag_skim_fat_anova,flag_whole_skim_anova,flag_whole_fat_anova,row_ID_kruskalwallis,k_value_kruskalwallis,p_value_k_kruskalwallis,mean_fat_kruskalwallis,mean_skim_kruskalwallis,mean_whole_kruskalwallis,sd_fat_kruskalwallis,sd_skim_kruskalwallis,sd_whole_kruskalwallis,mean_skim_fat_kruskalwallis,mean_whole_skim_kruskalwallis,mean_whole_fat_kruskalwallis,p_value_skim_fat_kruskalwallis,p_value_whole_skim_kruskalwallis,p_value_whole_fat_kruskalwallis,flag_skim_fat_kruskalwallis,flag_whole_skim_kruskalwallis,flag_whole_fat_kruskalwallis,merge_key
0,rowID_2020,184.073424,0.732373,0,0,0,0,0,0,0,0,0,rowID_2020,1.68432,0.239135,787473504.0,1.209770e+09,1.155680e+09,458074236.4,284173500.0,292628169.9,422296112.0,-54089872.0,368206240.0,0.262054,0.974677,0.348732,0,0,0,rowID_2020,3.038462,0.21888,787473504.0,1.209770e+09,1.155680e+09,458074236.4,284173500.0,292628169.9,422296112.0,-54089872.0,368206240.0,0.148915,0.563703,0.148915,0,0,0,0


In [32]:
for i in range(2, len(data_meta2)):
    
    # print progress
    if i%50 == 0:
        print(i/len(data_meta2))
        
    # search eligible row in library for row_i in original file
    d1_temp = data_lib[(data_lib['M/Z'] < data_meta2.row_m_z.iloc[i] + tol_mz) & 
             (data_lib['M/Z'] > data_meta2.row_m_z.iloc[i] - tol_mz) &
             (data_lib['Retention'] < data_meta2.row_retention_time.iloc[i] + tol_ret) &
             (data_lib['Retention'] > data_meta2.row_retention_time.iloc[i] - tol_ret)]
    
    # set merge key
    d1_temp['merge_key'] = i
    d2_temp = data_meta2.iloc[[i]]
    d2_temp['merge_key'] = i
    d1 = d1.append(d1_temp)
    d2 = d2.append(d2_temp)

0.07173601147776183
0.14347202295552366
0.2152080344332855
0.28694404591104733
0.3586800573888092
0.430416068866571
0.5021520803443329
0.5738880918220947
0.6456241032998565
0.7173601147776184
0.7890961262553802
0.860832137733142
0.9325681492109039


In [33]:
d1

,Garrett Lab ID,M/Z,Retention,Name,Formula,Molecular Weight,Adduct,Notes,Unnamed: 8,KEGG,Unnamed: 10,merge_key
240,1014p,184.073300,0.770,PHOSPHOCHOLINE,C5H14NO4P,184.150662,NaN,NaN,NaN,C00588,NaN,0
166,10917p,162.112500,1.010,L-CARNITINE,C7H15NO3,161.198900,NaN,NaN,NaN,C15025,NaN,2
269,NaN,210.061000,1.830,Citrate+NH3,NaN,NaN,M+NH3,NaN,NaN,NaN,NaN,4
16,305p,104.106900,0.860,Choline,C5H13NO,103.099700,NaN,Duffy archeology samples - Compound Discoverer,NaN,C00114,NaN,6
68,586p,132.076750,0.941,CREATINE,C4H9N3O2,131.133160,NaN,NaN,NaN,C00300,NaN,7
270,NaN,215.016000,1.830,Citrate+Na,NaN,NaN,M+Na,NaN,NaN,NaN,NaN,8
316,311p,215.016220,2.048,CITRATE,C6H8O7,192.123520,M+Na,NaN,NaN,C00158,NaN,8
271,NaN,129.018500,1.830,Citrate source fragment (+H and + NH3),NaN,NaN,source fragment,NaN,NaN,NaN,NaN,11
268,311p,193.034440,1.830,CITRATE,C6H8O7,192.123520,M+H,NaN,NaN,C00158,NaN,13
273,NaN,175.023800,1.830,Citrate source fragment (+NH3),NaN,NaN,source fragment,NaN,NaN,NaN,NaN,14


In [34]:
result = d1.merge(d2, on = ['merge_key'], how = 'inner')

In [35]:
result

,Garrett Lab ID,M/Z,Retention,Name,Formula,Molecular Weight,Adduct,Notes,Unnamed: 8,KEGG,Unnamed: 10,merge_key,row_ID,row_m_z,row_retention_time,flag_fat_threshold_005,flag_skim_threshold_005,flag_whole_threshold_005,flag_fat_threshold_100,flag_skim_threshold_100,flag_whole_threshold_100,flag_fat_threshold_200,flag_skim_threshold_200,flag_whole_threshold_200,row_ID_anova,f_value_anova,p_value_f_anova,mean_fat_anova,mean_skim_anova,mean_whole_anova,sd_fat_anova,sd_skim_anova,sd_whole_anova,mean_skim_fat_anova,mean_whole_skim_anova,mean_whole_fat_anova,p_value_skim_fat_anova,p_value_whole_skim_anova,p_value_whole_fat_anova,flag_skim_fat_anova,flag_whole_skim_anova,flag_whole_fat_anova,row_ID_kruskalwallis,k_value_kruskalwallis,p_value_k_kruskalwallis,mean_fat_kruskalwallis,mean_skim_kruskalwallis,mean_whole_kruskalwallis,sd_fat_kruskalwallis,sd_skim_kruskalwallis,sd_whole_kruskalwallis,mean_skim_fat_kruskalwallis,mean_whole_skim_kruskalwallis,mean_whole_fat_kruskalwallis,p_value_skim_fat_kruskalwallis,p_value_whole_skim_kruskalwallis,p_value_whole_fat_kruskalwallis,flag_skim_fat_kruskalwallis,flag_whole_skim_kruskalwallis,flag_whole_fat_kruskalwallis
0,1014p,184.073300,0.770,PHOSPHOCHOLINE,C5H14NO4P,184.150662,NaN,NaN,NaN,C00588,NaN,0,rowID_2020,184.073424,0.732373,0,0,0,0,0,0,0,0,0,rowID_2020,1.684320,0.239135,7.874735e+08,1.209770e+09,1.155680e+09,4.580742e+08,2.841735e+08,2.926282e+08,4.222961e+08,-5.408987e+07,3.682062e+08,0.262054,0.974677,0.348732,0,0,0,rowID_2020,3.038462,0.218880,7.874735e+08,1.209770e+09,1.155680e+09,4.580742e+08,2.841735e+08,2.926282e+08,4.222961e+08,-5.408987e+07,3.682062e+08,0.148915,0.563703,0.148915,0,0,0
1,10917p,162.112500,1.010,L-CARNITINE,C7H15NO3,161.198900,NaN,NaN,NaN,C15025,NaN,2,rowID_1973,162.112531,0.945414,0,0,0,0,0,0,0,0,0,rowID_1973,6.138617,0.020820,4.486682e+08,8.627042e+08,8.278780e+08,2.609883e+08,1.191339e+08,1.441874e+08,4.140361e+08,-3.482622e+07,3.792098e+08,0.028245,0.961999,0.042720,1,0,1,rowID_1973,4.884615,0.086960,4.486682e+08,8.627042e+08,8.278780e+08,2.609883e+08,1.191339e+08,1.441874e+08,4.140361e+08,-3.482622e+07,3.792098e+08,0.083265,0.772830,0.043308,0,0,1
2,NaN,210.061000,1.830,Citrate+NH3,NaN,NaN,M+NH3,NaN,NaN,NaN,NaN,4,rowID_2102,210.060849,1.830206,0,0,0,0,0,0,0,0,0,rowID_2102,11.726792,0.003115,1.748311e+08,3.741021e+08,3.945621e+08,7.925454e+07,8.211132e+07,4.532786e+07,1.992710e+08,2.046003e+07,2.197310e+08,0.008157,0.913131,0.004503,1,0,1,rowID_2102,7.538462,0.023070,1.748311e+08,3.741021e+08,3.945621e+08,7.925454e+07,8.211132e+07,4.532786e+07,1.992710e+08,2.046003e+07,2.197310e+08,0.020921,0.563703,0.020921,1,0,1
3,305p,104.106900,0.860,Choline,C5H13NO,103.099700,NaN,Duffy archeology samples - Compound Discoverer,NaN,C00114,NaN,6,rowID_2053,104.107469,0.879552,0,0,0,1,0,0,1,1,1,rowID_2053,2.459968,0.140515,2.981159e+08,5.241520e+08,5.016882e+08,8.216562e+07,2.089269e+08,1.588727e+08,2.260361e+08,-2.246379e+07,2.035723e+08,0.164608,0.978229,0.219718,0,0,0,rowID_2053,4.884615,0.086960,2.981159e+08,5.241520e+08,5.016882e+08,8.216562e+07,2.089269e+08,1.588727e+08,2.260361e+08,-2.246379e+07,2.035723e+08,0.043308,1.000000,0.083265,1,0,0
4,586p,132.076750,0.941,CREATINE,C4H9N3O2,131.133160,NaN,NaN,NaN,C00300,NaN,7,rowID_2012,132.076966,0.891071,0,0,0,0,0,0,0,0,0,rowID_2012,7.969972,0.010187,3.060638e+08,6.022965e+08,5.711609e+08,1.780364e+08,6.037563e+07,6.750942e+07,2.962328e+08,-3.113560e+07,2.650972e+08,0.013631,0.923454,0.024481,1,0,1,rowID_2012,6.615385,0.036601,3.060638e+08,6.022965e+08,5.711609e+08,1.780364e+08,6.037563e+07,6.750942e+07,2.962328e+08,-3.113560e+07,2.650972e+08,0.020921,0.248213,0.083265,1,0,0
5,NaN,215.016000,1.830,Citrate+Na,NaN,NaN,M+Na,NaN,NaN,NaN,NaN,8,rowID_2811,215.016095,1.851377,0,0,0,0,0,0,0,0,0,rowID_2811,6.674950,0.016686,1.215845e+08,2.297248e+08,2.128042e+08,4.204314e+07,4.841457e+07,4.439985e+07,1.081403e+08,-1.692065e+07,9.121968e+07,0.019559,0.858165,0.044627,1,0,1,rowID_2811,5.653846,0.059195,1.215845e+08,2.297248e+08,2.12804

In [36]:
result.to_csv('../results/allgroups_GLib_merged.csv', index = False, quoting = csv.QUOTE_ALL)

## Double check

In [57]:
data_lib.columns

Index(['row_ID', 'row m/z', 'row retention time', 'ID',
       'Identification method', 'Molecular formula', 'Name'],
      dtype='object')

In [59]:
data_meta1.columns

Index(['row_ID', 'f_value', 'p_value_f', 'mean_fat', 'mean_skim', 'mean_whole',
       'sd_fat', 'sd_skim', 'sd_whole', 'mean_skim_fat', 'mean_whole_skim',
       'mean_whole_fat', 'p_value_skim_fat', 'p_value_whole_skim',
       'p_value_whole_fat', 'flag_skim_fat', 'flag_whole_skim',
       'flag_whole_fat'],
      dtype='object')

In [60]:
data_meta2.columns

Index(['row_ID', 'row_m_z', 'row_retention_time', 'flag_fat_threshold_005',
       'flag_skim_threshold_005', 'flag_whole_threshold_005',
       'flag_fat_threshold_100', 'flag_skim_threshold_100',
       'flag_whole_threshold_100', 'flag_fat_threshold_200',
       'flag_skim_threshold_200', 'flag_whole_threshold_200', 'row_ID_anova',
       'f_value_anova', 'p_value_f_anova', 'mean_fat_anova', 'mean_skim_anova',
       'mean_whole_anova', 'sd_fat_anova', 'sd_skim_anova', 'sd_whole_anova',
       'mean_skim_fat_anova', 'mean_whole_skim_anova', 'mean_whole_fat_anova',
       'p_value_skim_fat_anova', 'p_value_whole_skim_anova',
       'p_value_whole_fat_anova', 'flag_skim_fat_anova',
       'flag_whole_skim_anova', 'flag_whole_fat_anova', 'row_ID_kruskalwallis',
       'k_value_kruskalwallis', 'p_value_k_kruskalwallis',
       'mean_fat_kruskalwallis', 'mean_skim_kruskalwallis',
       'mean_whole_kruskalwallis', 'sd_fat_kruskalwallis',
       'sd_skim_kruskalwallis', 'sd_whole_kruskalw

In [65]:
data_lib[data_lib['row m/z'] == data_meta2.row_m_z.iloc[0]]

,row_ID,row m/z,row retention time,ID,Identification method,Molecular formula,Name


In [66]:
for i in range(len(data_meta2)):
    if len(data_lib[data_lib['row m/z'] == data_meta2.row_m_z.iloc[i]]) != 0:
        print(data_meta2.row_ID.iloc[i])